<img src="images/dask_horizontal.svg" align="right" width="30%">

# 数组

<img src="images/array.png" width="25%" align="right">
Dask数组提供了一个并行的、大于内存的、使用阻塞算法的n维数组。简单地说：分布式Numpy。

* **并行**。使用你电脑上所有的核心。
* **大于内存**。 让你可以处理比可用内存更大的数据集，把你的数组分解成许多小块，按照最小化计算内存占用的顺序对这些小块进行操作，并有效地从磁盘上传输数据。
* **分块算法**。 通过执行许多较小的计算来执行大型计算。

在这个笔记本中，我们将通过从头开始实现一些阻塞算法来建立一些理解。
然后，我们将使用Dask Array来分析大型数据集，并行地使用熟悉的类似NumPy的API。

**相关文档**

* [Array documentation](https://docs.dask.org/en/latest/array.html)
* [Array screencast](https://youtu.be/9h_61hXCDuI)
* [Array API](https://docs.dask.org/en/latest/array-api.html)
* [Array examples](https://examples.dask.org/array.html)

## 创建数据

In [ ]:
%run prep.py -d random

## S设置

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

## 分块算法

在大型数据集上执行*分块算法*，是通过把它分解成许多小块来执行的。

例如，考虑取10亿个数字的总和，我们可以把数组分成1000块，每块大小为1,000,000。 我们可以将数组分解成1000个小块，每个小块的大小为1,000,000，取每个小块的和，然后取中间数的和。

我们通过执行许多较小的结果来实现预期的结果 (10 亿个数字的一个和) (每个百万个数字的 1000 个和，然后是 1000 个数字的另一个和)。

在下面的例子中，我们正是用 Python 和 NumPy 来实现这个目标。

In [ ]:
# 用h5py加载数据
# 将创建一个指向数据的指针，但并不实际加载。
import h5py
import os
f = h5py.File(os.path.join('data', 'random.hdf5'), mode='r')
dset = f['/x']

**使用分块算法计算总数**

在使用dask之前，我们先来考虑一下块状算法的概念。我们可以通过逐个分块加载来计算大量元素的和，并保持一个运行的总数。

在这里，我们通过以下方式来计算这个大数组在磁盘上的总和。

1.  计算阵列中每1,000,000个大小的块的总和。
2.  计算1,000个中间数的总和。

需要注意的是，这在笔记本内核中是一个连续的过程，包括加载和求和。

In [ ]:
# Compute sum of large array, one million numbers at a time
sums = []
for i in range(0, 1_000_000_000, 1_000_000):
    chunk = dset[i: i + 1_000_000]  # pull out numpy array
    sums.append(chunk.sum())

total = sum(sums)
print(total)

### 练习：用分块算法计算平均值

现在我们已经看到了上面的简单例子，试着做一个稍微复杂一点的问题。计算数组的平均数，暂时假设我们不知道数据中有多少元素。 你可以通过修改上面的代码，做如下改动来实现。

1.  计算每个块的和
2.  计算每个区块的长度
3.  计算1000个中间和与1000个中间长的和，并将其中一个除以另一个。

这种方法对于我们的情况来说是矫枉过正的，但如果我们事先不知道数组或单个块的大小，那么这种方法就会很好地通用。


In [ ]:
# Compute the mean of the array

In [ ]:
sums = []
lengths = []
for i in range(0, 1_000_000_000, 1_000_000):
    chunk = dset[i: i + 1_000_000]  # pull out numpy array
    sums.append(chunk.sum())
    lengths.append(len(chunk))

total = sum(sums)
length = sum(lengths)
print(total / length)

`dask.array`包含以下算法
--------------------------------------------

Dask.array是一个类似于NumPy的库，它可以做这些类型的技巧来操作不适合内存的大型数据集。 它超越了上面讨论的线性问题，扩展到了完整的N维算法和NumPy接口的一个不错的子集。

**创建`dask.array`对象**

你可以用`da.from_array`函数创建一个`dask.array``Array`对象。 该函数接受

1.  `data`。任何支持NumPy分片的对象，如`dset`。
2.  `chunks`。一个分块大小，用来告诉我们如何将数组分块，比如`(1_000_000,)`。

In [ ]:
import dask.array as da
x = da.from_array(dset, chunks=(1_000_000,))
x

**像处理numpy数组一样处理`dask.array`对象**

现在我们有了一个 "Array"，我们可以进行标准的numpy风格的计算，如算术、数学、切片、还原等。

界面很熟悉，但实际工作是不同的。`dask_array.sum()`和`numpy_array.sum()`做的事情不一样。

**有什么区别？**

`dask_array.sum()`建立一个计算的表达式。它还没有进行计算。`numpy_array.sum()`立即计算和。

*为什么会有区别？*

Dask数组被分割成块。每个分块必须明确地在该分块上运行计算。如果想要的答案来自整个数据集的一小块，那么在所有数据上运行计算会浪费CPU和内存。

In [ ]:
result = x.sum()
result

**Compute result**

Dask.array对象被懒惰地评估。 像`.sum`这样的操作会建立一个阻塞任务的图来执行。

我们通过调用`.compute()`来要求最终结果。 这将触发实际的计算。

In [ ]:
result.compute()

### 练习： 计算平均数

以及方差、标准差等。 这应该是上面例子的一个小变化。

看看Jupyter笔记本的tab-completion还可以做哪些操作。

这与你之前的结果相符吗？

性能和并行性
-------------------------------

<img src="images/fail-case.gif" width="40%" align="right">

在我们的第一个例子中，我们使用 "for "循环一次走过一个块的数组。 对于像 "sum "这样的简单操作，这是最好的。 然而对于复杂的操作，我们可能希望以不同的方式遍历数组。 特别是，我们可能需要以下操作。

1.  并行使用多个内核
2.  在进入下一个区块之前，对单个区块进行连锁操作。

`Dask.array`将您的数组操作翻译成一个相互关联的任务图，它们之间有数据依赖关系。 然后Dask会用多个线程并行执行这个图。 我们将在下一节讨论更多关于这个问题。



### 例

1.  构建一个20000x20000的正态分布随机值数组，分成1000x1000大小的块。
2.  沿一条轴线取平均值
3.  取每100个元素

In [ ]:
import numpy as np
import dask.array as da

x = da.random.normal(10, 0.1, size=(20000, 20000),   # 400 million element array 
                              chunks=(1000, 1000))   # Cut into 1000x1000 sized chunks
y = x.mean(axis=0)[::100]                            # Perform NumPy-style operations

In [ ]:
x.nbytes / 1e9  # Gigabytes of the input processed lazily

In [ ]:
%%time
y.compute()     # Time to compute the result

性能比较
---------------------------

以下实验是在一台沉重的个人笔记本电脑上进行的。 您的性能可能会有所不同。 如果你尝试NumPy版本，那么请确保你有超过4GB的主内存。

**NumPy: 19秒，需要千兆字节的内存**

```python
import numpy as np

%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
y

CPU times: user 19.6 s, sys: 160 ms, total: 19.8 s
Wall time: 19.7 s
```

**Dask数组：4秒，需要百万字节的内存**

```python
import dask.array as da

%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
y = x.mean(axis=0)[::100] 
y.compute() 

CPU times: user 29.4 s, sys: 1.07 s, total: 30.5 s
Wall time: 4.01 s
```

**Discussion**

注意，Dask数组计算运行时间为4秒，但使用了29.4秒的用户CPU时间。numpy计算运行时间为19.7秒，使用了19.6秒的用户CPU时间。

Dask完成得更快，但使用的总CPU时间更多，因为Dask由于分块大小，能够透明地进行并行计算。

*Questions*

* 如果dask chunks=(20000,20000)会怎样？
    * 计算会在4秒内运行吗？
    * 将会使用多少内存？
* 如果dask chunks=(25,25)会发生什么？
    * CPU和内存会怎样？

### 练习：气象数据

在`data/weather-big/*.hdf5`的HDF5文件中，有2GB的有些造作的天气数据。 我们将使用`h5py`库与这些数据进行交互，并使用`dask.array`对其进行计算。

我们的目标是可视化这个月地球表面的平均温度。 这将需要对所有这些数据进行平均。 我们将通过以下步骤来完成

1.  为磁盘上每一天的数据创建`h5py.Dataset`对象(`dsets`)。
2.  用 "da.from_array "调用来包装。
3.  调用 "da.stack "将这些数据集按时间堆叠起来。
4.  用`.mean()`方法计算沿新堆积的时间轴的平均值。
5.  用`matplotlib.pyplot.imshow`将结果可视化。

In [ ]:
%run prep.py -d weather

In [ ]:
import h5py
from glob import glob
import os

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename, mode='r')['/t2m'] for filename in filenames]
dsets[0]

In [ ]:
dsets[0][:5, :5]  # Slicing into h5py.Dataset object gives a numpy array

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 8))
plt.imshow(dsets[0][::4, ::4], cmap='RdBu_r');

**与`dask.array`**整合

使用`da.from_array`函数从`h5py.Dataset`对象列表中取出`dask.array`对象列表，大小为`(500，500)`。

In [ ]:
arrays = [da.from_array(dset, chunks=(500, 500)) for dset in dsets]
arrays

**用`da.stack`**将这个`dask.array`对象列表堆叠成一个`dask.array`对象。

沿着第一条轴堆叠这些数据，使所得数组的形状为`(31, 5760, 11520)`。

In [ ]:
x = da.stack(arrays, axis=0)
x

**沿时间(`0th`)轴绘制该数组的平均值**。

In [ ]:
# complete the following:
fig = plt.figure(figsize=(16, 8))
plt.imshow(..., cmap='RdBu_r')

In [ ]:
result = x.mean(axis=0)
fig = plt.figure(figsize=(16, 8))
plt.imshow(result, cmap='RdBu_r');

**绘制第一天与平均值的差异**。

In [ ]:
result = x[0] - x.mean(axis=0)
fig = plt.figure(figsize=(16, 8))
plt.imshow(result, cmap='RdBu_r');

###  练习： 子样本和存储

在上面的练习中，我们计算的结果很小，所以我们可以安全地调用`compute`。 有时我们的结果还是太大，无法放入内存，我们想把它保存到磁盘上。 在这种情况下，你可以使用以下两个函数之一。

1.  `da.store`。将dask.array存储到任何支持numpy setitem语法的对象中，如

        f = h5py.File('myfile.hdf5')
        output = f.create_dataset(shape=..., dtype=...)

        da.store(my_dask_array, output)

2.  `da.to_hdf5'。一个专门的函数，用于创建一个`dask.array`对象并将其存储到`HDF5`文件中。

        da.to_hdf5('data/myfile.hdf5', '/output', my_dask_array)

本练习的任务是**使用numpy步长切片对整个数据集在经纬度方向上进行2倍的子采样，然后使用上面列出的一个函数将这个结果存储到磁盘**。

提醒一下，Python slicing需要三个元素，即

    start:stop:step

    >>> L = [1, 2, 3, 4, 5, 6, 7]
    >>> L[::3]
    [1, 4, 7]

In [ ]:
# ...

In [ ]:
import h5py
from glob import glob
import os
import dask.array as da

filenames = sorted(glob(os.path.join('data', 'weather-big', '*.hdf5')))
dsets = [h5py.File(filename, mode='r')['/t2m'] for filename in filenames]

arrays = [da.from_array(dset, chunks=(500, 500)) for dset in dsets]

x = da.stack(arrays, axis=0)

result = x[:, ::2, ::2]

da.to_zarr(result, os.path.join('data', 'myfile.zarr'), overwrite=True)

## 例子：Lennard-Jones潜力 Lennard-Jones潜力

The [Lennard-Jones potential](https://en.wikipedia.org/wiki/Lennard-Jones_potential) is used in partical simuluations in physics, chemistry and engineering. It is highly parallelizable.

首先，我们将在7000个粒子上运行和配置文件Numpy版本。

In [ ]:
import numpy as np

# make a random collection of particles
def make_cluster(natoms, radius=40, seed=1981):
    np.random.seed(seed)
    cluster = np.random.normal(0, radius, (natoms,3))-0.5
    return cluster

def lj(r2):
    sr6 = (1./r2)**3
    pot = 4.*(sr6*sr6 - sr6)
    return pot

# build the matrix of distances
def distances(cluster):
    diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
    mat = (diff*diff).sum(-1)
    return mat

# the lj function is evaluated over the upper traingle
# after removing distances near zero
def potential(cluster):
    d2 = distances(cluster)
    dtri = np.triu(d2)
    energy = lj(dtri[dtri > 1e-6]).sum()
    return energy

In [ ]:
cluster = make_cluster(int(7e3), radius=500)

In [ ]:
%time potential(cluster)

请注意，最耗时的函数是`distances`。

In [ ]:
# this would open in another browser tab
# %load_ext snakeviz
# %snakeviz potential(cluster)

# alternative simple version given text results in this tab
%prun -s tottime potential(cluster)

### Dask version

这是Dask版本。只有`potential`函数需要重写，以便最好地利用Dask。

请注意，`da.nansum`已经用于整个$NxN$距离矩阵，以提高并行效率。


In [ ]:
import dask.array as da

# compute the potential on the entire
# matrix of distances and ignore division by zero
def potential_dask(cluster):
    d2 = distances(cluster)
    energy = da.nansum(lj(d2))/2.
    return energy

让我们将NumPy数组转换为Dask数组。由于整个NumPy数组都在内存中，所以按CPU核数来分块数组在计算上更有效率。

In [ ]:
from os import cpu_count

dcluster = da.from_array(cluster, chunks=cluster.shape[0]//cpu_count())

这一步应该很好地随着核心数量的增加而扩展。警告正在抱怨除以零，这就是为什么我们在`potential_dask`中使用`da.nansum`的原因。

In [ ]:
e = potential_dask(dcluster)
%time e.compute()

局限
-----------

Dask Array没有实现整个numpy接口。 期待这一点的用户会很失望。 值得注意的是，Dask Array有以下缺点。

1.  Dask没有实现所有的``np.linalg``。 这一点已经由一些优秀的BLAS/LAPACK实现完成，并且是众多正在进行的学术研究项目的重点。
2.  Dask Array不支持一些操作，在这些操作中，结果的形状取决于数组的值。对于那些它支持的操作（例如，用另一个布尔掩码掩盖一个Dask Array），块的大小将是未知的，这可能会导致其他的问题。   需要知道分块大小的操作。
3.  Dask Array不尝试像`sort``这样的操作，这些操作是臭名昭著的。  困难，而且在非常的情况下价值有所降低。  大数据（你很少真正需要一个完整的排序）。 通常，我们会包含像``topk``这样的并行友好的替代方案。
4.  Dask的开发是由即时需求驱动的，所以很多不太常用的    ``np.sometrue``等函数的实施并非纯粹出于对用户的保护。懒惰。 这些都会为社会做出很好的贡献。
    
* [Array documentation](https://docs.dask.org/en/latest/array.html)
* [Array screencast](https://youtu.be/9h_61hXCDuI)
* [Array API](https://docs.dask.org/en/latest/array-api.html)
* [Array examples](https://examples.dask.org/array.html)

In [ ]:
client.shutdown()